In [42]:
import torch
import torch.nn as nn
import torch.nn. functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

In [43]:
import zipfile
import os

def unzip_folder(zip_path, extract_to):
    """
    Unzip a zip archive to a specified directory.

    Parameters:
        zip_path (str): Path to the zip archive.
        extract_to (str): Directory where the contents will be extracted.

    Returns:
        None
    """
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

if __name__ == "__main__":
    # Specify the path to the zip archive and the directory to extract to
    zip_path = '/content/plus.zip'
    extract_to = '/content'

    # Create the extract directory if it doesn't exist
    os.makedirs(extract_to, exist_ok=True)

    # Unzip the folder
    unzip_folder(zip_path, extract_to)

    print(f"Successfully extracted contents to: {extract_to}")

Successfully extracted contents to: /content


In [44]:
transform=transforms.ToTensor()

In [55]:
import os
import pandas as pd
from skimage import io
import torch
from torch.utils.data import Dataset

class Red_dot(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.annotations = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotations)  # Number of samples in the dataset

    def __getitem__(self, index):
        img_path = os.path.join(self.root_dir, self.annotations.iloc[index, 0])
        image = io.imread(img_path)

        # Extract x and y coordinates as labels
        x_coord = float(self.annotations.iloc[index, 1])  # Assuming x coordinate is in column 1
        y_coord = float(self.annotations.iloc[index, 2])  # Assuming y coordinate is in column 2
        label = torch.tensor([x_coord, y_coord])  # Create a tensor with x and y coordinates

        if self.transform:
            image = self.transform(image)

        return image, label
root_dir='/content/plus/images/'
csv_file='/content/plus/plus_sign_coordinates.csv'

dataset=Red_dot(csv_file,root_dir,transform=transform)


In [56]:
train_set, test_set = torch.utils.data.random_split(dataset, [8000,2000])
train_loader = DataLoader(dataset=train_set, batch_size=32, shuffle=True)
test_loader = DataLoader(dataset=test_set, batch_size=32, shuffle=True)

In [57]:
# Grab 1 MNIST record/image
for i, (X_Train, y_train) in enumerate(train_loader):
    break


In [58]:
X_Train.shape


torch.Size([32, 4, 100, 100])

In [59]:
import torch
import torch.nn as nn
# Define CNN model using PyTorch
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(4, 32, kernel_size=3, padding=1)  # Adjust input channels to 4
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.relu3 = nn.ReLU()
        self.conv4 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.relu4 = nn.ReLU()
        self.conv5 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.relu5 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * 12 * 12, 128)  # Adjust the input size after convolutions
        self.relu5 = nn.ReLU()
        self.fc2 = nn.Linear(128, 32)
        self.relu6 = nn.ReLU()
        self.fc3 = nn.Linear(32, 8)
        self.relu7 = nn.ReLU()
        self.fc4 = nn.Linear(8, 2)

    def forward(self, x):
        x = self.relu1(self.conv1(x))
        x = self.pool1(x)
        x = self.relu2(self.conv2(x))
        x = self.pool2(x)
        x = self.relu3(self.conv3(x))
        x = self.relu4(self.conv4(x))
        x = self.relu5(self.conv5(x))
        x = self.pool3(x)
        x = self.flatten(x)
        x = self.relu5(self.fc1(x))
        x = self.relu6(self.fc2(x))
        x = self.relu7(self.fc3(x))
        coordinates = self.fc4(x)
        return coordinates
# Create an instance of the model
model = CNNModel()
print(model)


CNNModel(
  (conv1): Conv2d(4, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu1): ReLU()
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2): ReLU()
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu3): ReLU()
  (conv4): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu4): ReLU()
  (conv5): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu5): ReLU()
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=18432, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=32, bias=True)
  (relu6): ReLU()
  (fc3): Linear(in_features=32, out_features=8, bias=True)
  (re

In [60]:
# Create an instance of the model
model = CNNModel().to(device)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)

        # Compute loss (MSE loss for regression)
        loss = criterion(outputs, labels)

        # Backpropagation and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {running_loss / 100:.4f}')
            running_loss = 0.0

print('Finished Training')

Epoch [1/10], Step [100/250], Loss: 106493.7344
Epoch [1/10], Step [200/250], Loss: 923.8014
Epoch [2/10], Step [100/250], Loss: 385.6506
Epoch [2/10], Step [200/250], Loss: 332.7593
Epoch [3/10], Step [100/250], Loss: 315.8872
Epoch [3/10], Step [200/250], Loss: 303.4678
Epoch [4/10], Step [100/250], Loss: 221.0950
Epoch [4/10], Step [200/250], Loss: 227.8106
Epoch [5/10], Step [100/250], Loss: 221.8252
Epoch [5/10], Step [200/250], Loss: 231.8767
Epoch [6/10], Step [100/250], Loss: 260.1570
Epoch [6/10], Step [200/250], Loss: 253.1002
Epoch [7/10], Step [100/250], Loss: 197.0999
Epoch [7/10], Step [200/250], Loss: 212.1143
Epoch [8/10], Step [100/250], Loss: 215.4999
Epoch [8/10], Step [200/250], Loss: 302.1996
Epoch [9/10], Step [100/250], Loss: 221.9499
Epoch [9/10], Step [200/250], Loss: 769.4372
Epoch [10/10], Step [100/250], Loss: 508.2903
Epoch [10/10], Step [200/250], Loss: 142.5280
Finished Training


In [61]:
torch.save(model.state_dict(), 'trained_model.pth')
print('Model saved.')

Model saved.


In [69]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from PIL import Image

class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(4, 32, kernel_size=3, padding=1)  # Adjust input channels to 4
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.relu3 = nn.ReLU()
        self.conv4 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.relu4 = nn.ReLU()
        self.conv5 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.relu5 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * 12 * 12, 128)  # Adjust the input size after convolutions
        self.relu5 = nn.ReLU()
        self.fc2 = nn.Linear(128, 32)
        self.relu6 = nn.ReLU()
        self.fc3 = nn.Linear(32, 8)
        self.relu7 = nn.ReLU()
        self.fc4 = nn.Linear(8, 2)

    def forward(self, x):
        x = self.relu1(self.conv1(x))
        x = self.pool1(x)
        x = self.relu2(self.conv2(x))
        x = self.pool2(x)
        x = self.relu3(self.conv3(x))
        x = self.relu4(self.conv4(x))
        x = self.relu5(self.conv5(x))
        x = self.pool3(x)
        x = self.flatten(x)
        x = self.relu5(self.fc1(x))
        x = self.relu6(self.fc2(x))
        x = self.relu7(self.fc3(x))
        coordinates = self.fc4(x)
        return coordinates

# Load your trained model
model = CNNModel()
model.load_state_dict(torch.load('trained_model.pth'))  # Load the saved model

def preprocess_image(image_path):
    transform = transforms.Compose([
        transforms.Resize((100, 100)),  # Resize the image to match your input size
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])
    image = Image.open(image_path)

    # Convert the image to grayscale and then repeat to create 3 identical channels
    image = image.convert("L")  # Convert to grayscale
    image = transform(image)  # Convert to tensor and normalize

    # Repeat the single channel to create 3 channels (RGB)
    image = torch.cat([image, image, image,image], dim=0)  # Repeat grayscale to 3 channels

    image = image.unsqueeze(0)  # Add batch dimension
    return image


# Function to predict coordinates using the model
def predict_coordinates(model, image_tensor):
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        output = model(image_tensor)
        return output.squeeze()  # Return the predicted coordinates

# Path to your test image
test_image_path = '/content/plus_sign_0.png'

# Preprocess the image
input_image = preprocess_image(test_image_path)

# Make prediction
predicted_coordinates = predict_coordinates(model, input_image)

# Output the predicted coordinates
print(f"Predicted Coordinates (x, y): {predicted_coordinates[0].item()}, {predicted_coordinates[1].item()}")


Predicted Coordinates (x, y): -156.24560546875, 35.94917678833008
